## Guide for Creating Jupyter Notebook Kernal
1. Creating a Virtual Environment
First, you need to create a virtual environment. You can do this using venv (standard library in Python 3.3 and later) or virtualenv (if you are using an older version of Python or prefer it). Here, we'll use venv.

Open your terminal or command prompt and run the following commands:
```python -m venv env```

2. Activating the Virtual Environment
Before installing packages or running Python commands in your virtual environment, you need to activate it:

Windows ```.\env\Scripts\activate```

Mac ```source env/bin/activate```

3. Creating a Jupyter Kernel for the Virtual Environment

Install ipykernel if not already installed
```pip install ipykernel```

Create a Jupyter kernel for the virtual environment
```python -m ipykernel install --user --name=myenv --display-name="trulens kernal"```

Install Libraries

In [ ]:
#! pip install trulens_eval==0.22.2 chromadb==0.4.18 openai==1.3.7 ipython==8.21.0 ipywidgets==8.1.2

Add API keys
For this quickstart you will need an OpenAI Key.

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "<open-ai-key>"


Import from TruLens

In [2]:
# Create openai client
from openai import OpenAI
client = OpenAI()

# Imports main tools:
from trulens_eval import Feedback, OpenAI as fOpenAI, Tru
from trulens_eval import TruBasicApp
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


Create a simple chat completion function <br> A question answer bot 

In [6]:
def llm_open_ai(prompt):
    return client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", 
             "content": "You are a question and answer bot, \
             and you answer super upbeat."},
            {"role": "user", 
             "content": prompt}
        ]
    ).choices[0].message.content


Define prompt and call function

In [6]:
prompt_input= "What is photo synthesis?"
prompt_output = llm_open_ai(prompt_input)
prompt_output

"Yay, photosynthesis is like a magical process where plants use sunlight to make yummy food for themselves! Isn't nature amazing? 🌿🌞🍃"

## Feedback Functions

**Initialize Feedback Functions** <br>
Feed Back Function Guide
https://www.trulens.org/trulens_eval/feedback_function_guide/

In [12]:
# Initialize OpenAI-based feedback function collection class:
fopenai = fOpenAI()
# Define a relevance function from openai
f_relevance = Feedback(fopenai.relevance).on_input_output()
f_relevance_qs = Feedback(fopenai.qs_relevance).on_input_output()

✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to __record__.main_output or `Select.RecordOutput` .


Want to know more feedback functions? <br>
visit there [git](https://github.com/truera/trulens/tree/main/trulens_eval) <br> trulens_eval/trulens_eval/feedback/v2/feedback.py

Instrument the callable for logging with TruLens

In [13]:
from trulens_eval import TruBasicApp
tru_llm_standalone_recorder = TruBasicApp(llm_open_ai, app_id="trulens_demo", 
                                          feedbacks=[f_relevance, f_relevance_qs])

In [14]:
with tru_llm_standalone_recorder as recording:
    tru_llm_standalone_recorder.app(prompt_input)


In [15]:
tru.get_records_and_feedback(app_ids=[])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,...,Input Sentiment_calls,Coherence_calls,Output Sentiment_calls,relevance,qs_relevance,relevance_calls,qs_relevance_calls,latency,total_tokens,total_cost
0,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_29af9eb6e77f2c86e2a877b750219690,"""What types of insurance are commonly used to ...","""There are several types of insurance commonly...",-,"{""record_id"": ""record_hash_29af9eb6e77f2c86e2a...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T13:30:42.272951"", ""...",...,[{'args': {'text': 'What types of insurance ar...,[{'args': {'text': 'There are several types of...,[{'args': {'text': 'There are several types of...,NaN,NaN,NaN,NaN,2,113,0.000207
1,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_09fb2e694ff895fb9117e6a645ac0c69,"""\u00bfCu\u00e1l es la diferencia entre un seg...","""\u00a1Hola! Claro, la principal diferencia en...",-,"{""record_id"": ""record_hash_09fb2e694ff895fb911...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T13:30:44.517324"", ""...",...,[{'args': {'text': '¿Cuál es la diferencia ent...,"[{'args': {'text': '¡Hola! Claro, la principal...","[{'args': {'text': '¡Hola! Claro, la principal...",NaN,NaN,NaN,NaN,3,200,0.000379
2,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_1e49e968e7db71646b9b73f771f757ad,"""Comment fonctionne l'assurance automobile en ...","""Oh la la, quelle question importante! En cas ...",-,"{""record_id"": ""record_hash_1e49e968e7db71646b9...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T13:30:47.679968"", ""...",...,[{'args': {'text': 'Comment fonctionne l'assur...,"[{'args': {'text': 'Oh la la, quelle question ...","[{'args': {'text': 'Oh la la, quelle question ...",NaN,NaN,NaN,NaN,1,140,0.000260
3,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_3e94393adf4451cb42f32400fdff062e,"""Welche Arten von Versicherungen sind in Deuts...","""Hallo! In Deutschland gibt es verschiedene Ar...",-,"{""record_id"": ""record_hash_3e94393adf4451cb42f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T13:30:49.925433"", ""...",...,[{'args': {'text': 'Welche Arten von Versicher...,[{'args': {'text': 'Hallo! In Deutschland gibt...,[{'args': {'text': 'Hallo! In Deutschland gibt...,NaN,NaN,NaN,NaN,2,173,0.000324
4,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_5ce9d92c85b0c3f5142d70ec3340cf7d,"""\u4fdd\u9669\u5982\u4f55\u4fdd\u62a4\u8d22\u4...","""\u54c7\uff01\u4fdd\u9669\u771f\u662f\u4e2a\u8...",-,"{""record_id"": ""record_hash_5ce9d92c85b0c3f5142...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T13:30:52.193688"", ""...",...,"[{'args': {'text': '保险如何保护财产损失？'}, 'ret': 1.0,...",[{'args': {'text': '哇！保险真是个超级有用的东西！如果您的财产遭受损失，...,[{'args': {'text': '哇！保险真是个超级有用的东西！如果您的财产遭受损失，...,NaN,NaN,NaN,NaN,2,169,0.000318
5,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_6e49d0e272669f6897d733617d398486,"""\u041a\u0430\u043a\u043e\u0432\u044b \u043e\u...","""\u041f\u0440\u0435\u043a\u0440\u0430\u0441\u0...",-,"{""record_id"": ""record_hash_6e49d0e272669f6897d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T13:30:54.906033"", ""...",...,[{'args': {'text': 'Каковы основные виды страх...,[{'args': {'text': 'Прекрасный вопрос! В Росси...,[{'args': {'text': 'Прекрасный вопрос! В Р

Scores

In [12]:
tru.get_records_and_feedback(app_ids=[])[0][['input','output','relevance','qs_relevance']]

,input,output,relevance,qs_relevance
0,"""What is photo synthesis?""","""Oh, I love talking about photosynthesis! Phot...",0.2,0.8


Complete Results

In [22]:
tru.get_records_and_feedback(app_ids=[])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,...,Output Sentiment,Input Sentiment,Coherence,Language Match_calls,Output Sentiment_calls,Input Sentiment_calls,Coherence_calls,latency,total_tokens,total_cost
0,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_2d4d092b23aaeab4d9e4581b10f35216,"""What types of insurance are commonly used to ...","""Oh, great question! There are a few types of ...",-,"{""record_id"": ""record_hash_2d4d092b23aaeab4d9e...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-20T13:36:27.451358"", ""...",...,1.0,1.0,0.8,[],"[{'args': {'text': 'Oh, great question! There ...",[{'args': {'text': 'What types of insurance ar...,"[{'args': {'text': 'Oh, great question! There ...",2,107,0.000195
1,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_b67ad05c11db59d8826e97c6fbb33cc7,"""\u00bfCu\u00e1l es la diferencia entre un seg...","""\u00a1Hola! \u00a1Qu\u00e9 excelente pregunta...",-,"{""record_id"": ""record_hash_b67ad05c11db59d8826...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-20T13:36:29.630775"", ""...",...,1.0,0.0,0.9,[],[{'args': {'text': '¡Hola! ¡Qué excelente preg...,[{'args': {'text': '¿Cuál es la diferencia ent...,[{'args': {'text': '¡Hola! ¡Qué excelente preg...,3,160,0.000299
2,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_e4a099717eb4c239b79709a3d6b94a8d,"""Comment fonctionne l'assurance automobile en ...","""G\u00e9nial que tu poses cette question! En c...",-,"{""record_id"": ""record_hash_e4a099717eb4c239b79...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-20T13:36:33.466885"", ""...",...,1.0,1.0,0.8,[],[{'args': {'text': 'Génial que tu poses cette ...,[{'args': {'text': 'Comment fonctionne l'assur...,[{'args': {'text': 'Génial que tu poses cette ...,4,207,0.000395
3,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_341641d52489f2768fcb8e7604fd86cb,"""Welche Arten von Versicherungen sind in Deuts...","""Yay, Versicherungen sind so wichtig, um uns z...",-,"{""record_id"": ""record_hash_341641d52489f2768fc...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-20T13:36:37.611090"", ""...",...,1.0,0.0,0.2,[],"[{'args': {'text': 'Yay, Versicherungen sind s...",[{'args': {'text': 'Welche Arten von Versicher...,"[{'args': {'text': 'Yay, Versicherungen sind s...",2,129,0.000236
4,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_36dc21b45ab7677f3fd8516e6b302f8b,"""\u4fdd\u9669\u5982\u4f55\u4fdd\u62a4\u8d22\u4...","""\u4fdd\u9669\u662f\u4e00\u79cd\u5f88\u68d2\u7...",-,"{""record_id"": ""record_hash_36dc21b45ab7677f3fd...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-20T13:36:40.225540"", ""...",...,1.0,1.0,0.2,[],[{'args': {'text': '保险是一种很棒的方式来保护财产损失！当你购买保险时，...,"[{'args': {'text': '保险如何保护财产损失？'}, 'ret': 1.0,...",[{'args': {'text': '保险是一种很棒的方式来保护财产损失！当你购买保险时，...,3,180,0.000340
5,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_b0c75c7ebfdc0a07e0df513e63d74be5,"""\u041a\u0430\u043a\u043e\u0432\u044b \u043e\u...","""\u041f\u0440\u0435\u043a\u0440\u0430\u0441\u0...",-,"{""record_id"": ""record_hash_b0c75c7ebfdc0a07e0d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-20T13:36:43.544628"", ""...",...,1.0,1.0,0.2,[],[{'args': {'text': 'Прекрасный вопрос! Основны...,[{'args': {'text': 'Каковы основные виды страх...,[{'args': {'text': 'Прекра

A dashboard?


In [16]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.0.170:8501



<Popen: returncode: -11 args: ['streamlit', 'run', '--server.headless=True',...>

### Don't want to use recorder ?

In [16]:
prompt_input = "What is the process of photosynthesis?"
Relevant_Answer = "Photosynthesis is a process used by plants, algae, and certain bacteria to convert light energy into chemical energy stored in glucose or other sugars. This process involves the absorption of carbon dioxide (CO2), water (H2O), and light energy, primarily from the sun. Through a series of reactions known as the light-dependent and light-independent (Calvin cycle) reactions, these inputs are transformed into glucose and oxygen (O2). The overall chemical equation for photosynthesis can be summarized as 6CO2 + 6H2O + light energy → C6H12O6 + 6O2."
Irrelevant_Answer = "Photosynthesis is a method used by animals to digest food. It involves the stomach and intestines breaking down food into nutrients that the body can use for energy, growth, and repair. This process is crucial for the survival of animals and requires a complex digestive system."
print('Good Answer',fopenai.relevance(prompt_input, Relevant_Answer))
print('Bad Answer',fopenai.relevance(prompt_input, Irrelevant_Answer))

Good Answer 1.0
Bad Answer 0.0


In [17]:
prompt_input = "What is the process of photosynthesis?"
Relevant_Context = "I'm teaching a group of 8th-grade students about plant biology, and I need to explain how plants make their own food. What is the process of photosynthesis?"
Irrelevant_Context = "I've always been fascinated by how different plant species adapt to their environments. What is the process of photosynthesis?"
print('Good Answer',fopenai.qs_relevance(prompt_input, Relevant_Context))
print('Bad Answer',fopenai.qs_relevance(prompt_input, Irrelevant_Context))

Good Answer 1.0
Bad Answer 0.2


### Other Evaluation Metrics (Helpful)

In [3]:
from trulens_eval import Feedback
from trulens_eval.feedback.provider import OpenAI
from trulens_eval.feedback.provider import Huggingface
# Initialize provider classes
provider = OpenAI()
hugs_provider = Huggingface()

# LLM-based feedback functions
f_coherence = Feedback(
    provider.coherence_with_cot_reasons, name="Coherence"
    ).on_output()

f_input_sentiment = Feedback(
    provider.sentiment_with_cot_reasons, name="Input Sentiment"
    ).on_input()

f_output_sentiment = Feedback(
    provider.sentiment_with_cot_reasons, name="Output Sentiment"
    ).on_output()
        
f_langmatch = Feedback(
    hugs_provider.language_match, name="Language Match"
    ).on_input_output()

helpful_feedbacks = [
    f_coherence,
    f_input_sentiment,
    f_output_sentiment,
    f_langmatch,
    ]

✅ In Coherence, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Input Sentiment, input text will be set to __record__.main_input or `Select.RecordInput` .
✅ In Output Sentiment, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Language Match, input text1 will be set to __record__.main_input or `Select.RecordInput` .
✅ In Language Match, input text2 will be set to __record__.main_output or `Select.RecordOutput` .


In [4]:
helpful_evals = [
    "What types of insurance are commonly used to protect against property damage?",
    "¿Cuál es la diferencia entre un seguro de vida y un seguro de salud?",
    "Comment fonctionne l'assurance automobile en cas d'accident?",
    "Welche Arten von Versicherungen sind in Deutschland gesetzlich vorgeschrieben?",
    "保险如何保护财产损失？",
    "Каковы основные виды страхования в России?",
    "ما هو التأمين على الحياة وما هي فوائده؟",
    "自動車保険の種類とは何ですか？",
    "Como funciona o seguro de saúde em Portugal?",
    "बीमा क्या होता है और यह कितने प्रकार का होता है?"
]

In [7]:
tru_llm_standalone_recorder = TruBasicApp(llm_open_ai, app_id="Training_20Feb_helpful", feedbacks=helpful_feedbacks)

In [8]:
tru.reset_database()
with tru_llm_standalone_recorder as recording:
    for prompt_input in helpful_evals:
        tru_llm_standalone_recorder.app(prompt_input)

App Training_20Feb_helpful was not present in database. Adding it.


In [ ]:
tru.get_records_and_feedback(app_ids=[])[0][['input','output','Language Match','Input Sentiment','Output Sentiment','Coherence']] 

In [10]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.0.170:8501



<Popen: returncode: -11 args: ['streamlit', 'run', '--server.headless=True',...>